# 2.5. Some advanced neural network approaches

## Data loading and preprocessing

In [1]:
import pandas as pd
from google.colab import files
#uploaded = files.upload()
data = pd.read_csv('/content/preprocessed_data.csv')
data = data.sample(frac=1) # a bit weird
data.head()

,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,...,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
2722,Red,False,3,2.0,0.0,0.0,3.000,1.000,8.000000,2.50,13.500000,4.0,18.000,11.500000,35.500,16.500,0.000000,1.000000,0.500000,0.50,1.50,39.500000,18.000000,0.35,1.000000,0.500000,0.000,0.000000,60.000000,36.000000,0.0,2.0,10.50,8.000000,5.500000,3.000000,16.500,6.500000,30.500000,19.500000,...,111.000000,19.333333,9.0,900.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,3.0,182.88,182.88,240.0,34.0,30.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
854,Red,False,5,0.0,4.0,0.0,5.000,3.500,6.250000,4.00,14.500000,3.5,38.750,21.500000,52.000,23.500,0.000000,2.500000,2.000000,4.75,0.25,59.500000,29.000000,0.50,1.000000,3.000000,2.000,0.790000,130.750000,90.000000,4.0,0.0,5.25,3.750000,5.500000,3.250000,13.250,5.500000,3.000000,2.500000,...,171.666667,93.666667,11.0,1100.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,165.10,167.64,125.0,27.0,28.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
2338,Blue,True,5,0.0,3.0,0.0,17.000,12.000,1.333333,1.00,154.333333,59.0,6.000,2.333333,129.000,39.000,0.333333,15.666667,11.333333,1.00,0.00,161.666667,62.333333,0.41,0.333333,2.666667,2.000,0.283333,187.333333,87.666667,3.0,0.0,13.00,9.666667,1.333333,0.666667,107.000,36.333333,0.666667,0.333333,...,132.000000,44.000000,10.0,929.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,167.64,170.18,135.0,25.0,33.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1795,Blue,False,3,1.0,0.0,0.0,17.000,9.000,41.000000,26.00,202.000000,53.0,0.000,0.000000,206.000,50.000,0.000000,20.000000,20.000000,0.00,0.00,243.000000,79.000000,0.32,0.000000,5.000000,0.000,0.000000,265.000000,100.000000,0.0,1.0,61.00,48.000000,105.000000,84.000000,259.000,112.000000,6.000000,6.000000,...,187.000000,81.500000,6.0,900.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,180.34,185.42,135.0,31.0,28.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
67,Blue,False,3,1.0,0.0,0.0,12.375,7.625,5.000000,2.75,135.250000,53.0,6.625,4.625000,122.125,42.125,0.000000,12.375000,10.625000,1.50,0.00,146.875000,60.375000,0.40,0.125000,5.625000,1.875,0.335000,183.375000,90.250000,4.0,3.0,16.25,10.250000,3.500000,2.375000,127.125,47.625000,5.875000,3.500000,...,97.250000,43.250000,9.0,628.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,3.0,187.96,193.04,170.0,30.0,31.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0


In [0]:
data['Winner'] = data['Winner'].map(lambda x: 1 if x == 'Red' else 0)
data['title_bout'] = data['title_bout'].map(lambda x: 1 if x == 'True' else 0)

In [0]:
train_size = int(0.8*len(data))
features = data.drop(columns=['Winner'])
targets = data['Winner']
X_train, X_test = features.values[:train_size, :], features.values[train_size:, :]
y_train, y_test = targets.values[:train_size], targets.values[train_size:]

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
#sns.pairplot(data)

In [0]:
corr = data.corr()
cmap = sns.diverging_palette(250, 10, as_cmap=True)
plt.figure(figsize=(40, 80))
sns.heatmap(corr[['Winner']], cmap=cmap, vmax=.3,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},
            annot=True)

## Tensorflow ANNs

In [50]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'), # 0 or x: _____/
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid') # [0, 1]
])

red = len(y_train[y_train>0])
blue = len(y_train) - red
total = len(y_train)
weight_for_red = total / (2 * red) 
weight_for_blue = total / (2 * blue)
class_weight = {0: weight_for_blue, 1: weight_for_red}
print(class_weight)

adam_optimizer = tf.keras.optimizers.Adam()
model.compile(
    optimizer=adam_optimizer,
    loss='binary_crossentropy',
    metrics=[
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
    ]
)

save_best_callback = tf.keras.callbacks.ModelCheckpoint(
    '/content/model-{epoch:02d}-{val_accuracy:.2f}.hdf5', 
    monitor='val_accuracy', 
    verbose=1,
    save_best_only=True, 
    save_weights_only=False,
    save_frequency=1)

model.fit(X_train, 
          y_train,
          class_weight=class_weight,
          batch_size=64,
          validation_split=0.1,
          callbacks=[save_best_callback],
          epochs=50)

{0: 1.5010449320794148, 1: 0.7497390396659708}
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 2585 samples, validate on 288 samples
Epoch 1/50
2048/2585 [======================>.......] - ETA: 0s - loss: 11.0334 - tp: 706.0000 - fp: 329.0000 - tn: 360.0000 - fn: 653.0000 - accuracy: 0.5205 - precision: 0.6821 - recall: 0.5195 - auc: 0.5246
Epoch 00001: val_accuracy improved from -inf to 0.50694, saving model to /content/model-01-0.51.hdf5
2585/2585 [==============================] - 3s 1ms/sample - loss: 10.8084 - tp: 882.0000 - fp: 424.0000 - tn: 443.0000 - fn: 836.0000 - accuracy: 0.5126 - precision: 0.6753 - recall: 0.5134 - auc: 0.5151 - val_loss: 3.7268 - val_tp: 82.0000 - val_fp: 26.0000 - val_tn: 64.0000 - val_fn: 116.0000 - val_accuracy: 0.5069 - val_precision: 0.7593 - val_recall: 0.4141 - val_auc: 0.5750
Epoch 2/50
1920/2585 [=====================>........] - ETA: 0s - loss: 6.8153 - tp: 678.0000 - fp: 300.0000 - tn: 352.0000 - fn: 590.0000 - accuracy: 0.5365 - pr

In [52]:
import numpy as np
model = tf.keras.models.load_model('/content/model-43-0.64.hdf5')
model.evaluate(X_test, y_test)
#np.round(model.predict(X_test))

719/719 [==============================] - 1s 1ms/sample - loss: 0.6553 - tp: 285.0000 - fp: 101.0000 - tn: 154.0000 - fn: 179.0000 - accuracy: 0.6106 - precision: 0.7383 - recall: 0.6142 - auc: 0.6447


[0.6552500779506063,
 285.0,
 101.0,
 154.0,
 179.0,
 0.61057025,
 0.738342,
 0.61422414,
 0.64474726]

In [0]:
!pip install --upgrade tensorflow